In [3]:
from Idntimes import Idntimes
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
obj = Idntimes()

In [8]:
end_date = date(2018, 9, 1)
start_date = date(2018, 8, 28)
init_page = 1

In [9]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], 'news', init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-08-31
page  1
https://www.idntimes.com/ajax/index?category=news&type=all&page=1&date=2018-08-31
https://www.idntimes.com/news/indonesia/afrianisusanti/bawaslu-diminta-transparan-ungkap-kasus-mahar-politik-sandiaga
memasukkan berita id  1535648517
Insert berita  Bawaslu Diminta Transparan Ungkap Kasus Mahar Politik Sandiaga
masuk
https://www.idntimes.com/news/indonesia/teatrika/ditantang-debat-sekjen-pkb-soal-perekonomian-begini-reaksi-sandiaga
memasukkan berita id  1535648516
Insert berita  Ditantang Debat Sekjen PKB soal Perekonomian, Begini Reaksi Sandiaga
masuk
https://www.idntimes.com/news/indonesia/vanny-rahman/guru-besar-uin-padang-yang-meninggal-di-persidangan-dikenal-jujur
memasukkan berita id  1535648518
Insert berita  Guru Besar UIN Padang yang Meninggal di Persidangan Dikenal Jujur
masuk
https://www.idntimes.com/news/indonesia/lucia-vania/mulai-1-september-rute-ka-wijaya-kusuma-hingga-banyuwangi
memasukkan berita id  1535648509
Insert berita  Mulai 1 September, Rute KA 

KeyboardInterrupt: 